In [103]:
import random
import math
import statistics

class TableEntry:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.in_table = True

    def get_key(self):
        return self.key

    def get_value(self):
        return self.value

    def set_value(self, value):
        self.value = value

    def is_in_table(self):
        return self.in_table

    def set_to_removed(self):
        self.key = None
        self.value = None
        self.in_table = False

    def is_removed(self):
        return (self.key is None) and (self.value is None) and not self.in_table

In [104]:
class HashTableWithCount:
    def __init__(self, initial_capacity=307):
        self.table = [None] * initial_capacity
        self.size = 0
        self.probe_count = 0
    
    def get_next_prime(self, n):
        if n <= 2:
            return 2
        prime = n
        if prime % 2 == 0:
            prime += 1
        while not self.is_prime(prime):
            prime += 2
        return prime

    def is_prime(self, n):
        if n <= 1:
            return False
        for i in range(2, int(math.sqrt(n)) + 1):
            if n % i == 0:
                return False
        return True
    
    def check_size(self, size):
        pass

    def get_hash_index(self, key):
        return hash(key) % len(self.table)

    def is_hash_table_too_full(self):
        pass

    def enlarge_hash_table(self):
        pass

    def reset_probe_count(self):
        self.probe_count = 0
    
    def get_probes(self):
        return self.probe_count

In [105]:
class LinearProbingWithCount(HashTableWithCount):
    def __init__(self):
        super().__init__()
        
    def probe(self, index, key):
        for i in range(len(self.table)):
            bucket_index = (index + i) % len(self.table)
            self.probe_count += 1
            if self.table[bucket_index] is None:
                return -1
            elif self.table[bucket_index].is_in_table() and self.table[bucket_index].key == key:
                return bucket_index
        return index
    
    def locate(self, index, key):   # Optional per Eric
        for i in range(len(self.table)):
            bucket_index = (index + i) % len(self.table)
            if self.table[bucket_index] is None:
                return -1
            elif self.table[bucket_index].is_in_table() and self.table[bucket_index].get_key() == key:
                return bucket_index
        return -1
    
    def add(self, key, value):
        new_node = TableEntry(key, value)
        index = self.get_hash_index(key)
        for i in range(len(self.table)):
            bucket_index = (index + i) % len(self.table)
            if self.table[bucket_index] is None:
                self.table[bucket_index] = new_node
                self.size += 1
                return
            elif self.table[bucket_index].is_removed():
                placeholder_index = bucket_index
                for j in range(i + 1, len(self.table)):
                    next_index = (index + j) % len(self.table)
                    if self.table[next_index] is None:
                        self.table[placeholder_index] = new_node
                        self.size += 1
                        return
                    elif self.table[next_index].is_in_table() and self.table[next_index].get_key() == key:
                        self.table[next_index].set_value(value)
                        return
            elif self.table[bucket_index].is_in_table() and self.table[bucket_index].get_key() == key:
                self.table[bucket_index].set_value(value)
                return

    def remove(self, key):
        index = self.locate(self.get_hash_index(key), key)
        if index != -1:
            self.table[index].set_to_removed()
            self.size -= 1

    def get_value(self, key):
        index = self.locate(self.get_hash_index(key), key)
        if index != -1:
            return self.table[index].get_value()

    def contains(self, key):
        index = self.locate(self.get_hash_index(key), key)
        if index != -1:
            return True
        return False

    def is_empty(self):
        return self.size == 0
    
    def get_size(self):
        return self.size

    def clear(self):
        self.table = [None] * len(self.table)
        self.size = 0
        self.reset_probe_count()



In [106]:
class DoubleHashingWithCount(HashTableWithCount):
    def __init__(self):
        super().__init__()
        
    def primary_hash(self, key):
        """First hash function."""
        return hash(key) % len(self.table)
    
    def secondary_hash(self, key):
        """Second hash function to calculate step size."""
        return 1 + (hash(key) % (len(self.table) - 1))

    def probe(self, index, key):
        for i in range(len(self.table)):
            bucket_index = self.secondary_hash(index + i)
            self.probe_count += 1
            if self.table[bucket_index] is None:
                return -1
            elif self.table[bucket_index].is_in_table() and self.table[bucket_index].key == key:
                return bucket_index
        return index

    def locate(self, index, key):   # Optional per Eric
        for i in range(len(self.table)):
            bucket_index = self.secondary_hash(index + i)
            if self.table[bucket_index] is None:
                return -1
            elif self.table[bucket_index].is_in_table() and self.table[bucket_index].get_key() == key:
                return bucket_index
        return -1
    
    def add(self, key, value):
        new_node = TableEntry(key, value)
        index = self.primary_hash(key)
        for i in range(len(self.table)):
            bucket_index = self.secondary_hash(index + i)
            if self.table[bucket_index] is None:
                self.table[bucket_index] = new_node
                self.size += 1
                return
            elif self.table[bucket_index].is_removed():
                placeholder_index = bucket_index
                for j in range(i + 1, len(self.table)):
                    next_index = (index + j) % len(self.table)
                    if self.table[next_index] is None:
                        self.table[placeholder_index] = new_node
                        self.size += 1
                        return
                    elif self.table[next_index].is_in_table() and self.table[next_index].get_key() == key:
                        self.table[next_index].set_value(value)
                        return
            elif self.table[bucket_index].is_in_table() and self.table[bucket_index].get_key() == key:
                self.table[bucket_index].set_value(value)
                return

    def remove(self, key):
        index = self.locate(self.primary_hash(key), key)
        if index != -1:
            self.table[index].set_to_removed()
            self.size -= 1

    def get_value(self, key):
        index = self.locate(self.primary_hash(key), key)
        if index != -1:
            return self.table[index].get_value()

    def contains(self, key):
        index = self.locate(self.primary_hash(key), key)
        if index != -1:
            return True
        return False

    def is_empty(self):
        return self.size == 0
    
    def get_size(self):
        return self.size
    
    def clear(self):
        self.table = [None] * len(self.table)
        self.size = 0
        self.reset_probe_count()

In [107]:
class GetStatistics:
    
    def main():
        repetitions = 1000
        a_names = GetStatistics.get_1000_names()
        b_names = GetStatistics.get_10000_names()
        linear_ht = LinearProbingWithCount()
        double_ht = DoubleHashingWithCount()
        linear_totals, double_hashing_totals = [], []
        for _ in range(repetitions):
            linear_ht.clear()
            double_ht.clear()

            GetStatistics.choose_100_names_and_add(linear_ht, double_ht, a_names)
            
            for _ in range(100):
                random_b_name = b_names[random.randint(0, 9999)]
                linear_ht.probe(linear_ht.get_hash_index(random_b_name), random_b_name)
                double_ht.probe(linear_ht.get_hash_index(random_b_name), random_b_name)
            linear_totals.append(linear_ht.get_probes())
            double_hashing_totals.append(double_ht.get_probes())
        print(f"Linear Probing totals per 100 trials over {repetitions} repetitions: {linear_totals}")
        print(f"Linear Probing total probes for {repetitions} trials: {sum(linear_totals) / len(linear_totals)}")
        print(f"Standard deviation for Linear Probing over {repetitions} trials: {round(statistics.stdev(linear_totals), 3)}")
        print('')
        print(f"Double Hashing totals per 100 trials over {repetitions} repetitions: {double_hashing_totals}")
        print(f"Double Hashing average probes for {repetitions} trials: {sum(double_hashing_totals) / len(double_hashing_totals)}")
        print(f"Standard deviation for Double Hashing over 1000 trials: {round(statistics.stdev(double_hashing_totals), 3)}")
        

    def choose_100_names_and_add(linear_ht, double_ht, names):
        for _ in range(100):
                random_a_name = names[random.randint(0, 999)]
                linear_ht.add(random_a_name, random_a_name)
                double_ht.add(random_a_name, random_a_name)

    def get_1000_names():
        names = []
        names = GetStatistics.fill_list(names, 'a', 1000)
        return names

    def get_10000_names():
        names = []
        names = GetStatistics.fill_list(names, 'b', 10000)
        return names
    
    # Filling sets first to ensure uniqueness, then converting to lists
    def fill_list(name_list, starting_letter, target_number):
        name_set = set()
        while len(name_set) < target_number:
            # All words in set_a start with specified starting letter followed by 6 random digits
            word = starting_letter
            for _ in range(6):
                word += str(random.randint(0, 9))
            name_set.add(word)
        name_list = list(name_set)
        return name_list

if __name__ == "__main__":
    # A hash table with 100 entries needs a size of approximately 342 in order to average 1.5 comparisons
    # for unsuccessful searches using linear probing. The next largest prime is 347.
    # Using double hashing, a size of 300 is needed. The next largest prime is 307.
    # For this experiment we will set a default size of 307 in the HashTableWithCount constructor so that 
    # we can compare both methods on the same table size.
    GetStatistics.main()



Linear Probing totals per 100 trials over 1000 repetitions: [149, 171, 148, 131, 146, 159, 158, 154, 154, 158, 154, 147, 162, 151, 145, 158, 169, 167, 165, 145, 151, 149, 163, 217, 150, 138, 155, 140, 150, 149, 165, 153, 148, 163, 137, 142, 159, 140, 167, 159, 145, 177, 161, 161, 154, 169, 140, 162, 159, 157, 160, 148, 146, 163, 156, 155, 137, 152, 143, 163, 169, 157, 164, 140, 151, 162, 150, 158, 160, 144, 163, 160, 192, 152, 165, 175, 152, 159, 147, 142, 141, 133, 160, 151, 184, 157, 143, 147, 158, 161, 164, 161, 142, 164, 161, 144, 148, 160, 148, 162, 147, 218, 135, 156, 155, 144, 132, 148, 168, 129, 151, 143, 151, 166, 154, 167, 143, 158, 153, 153, 145, 151, 149, 147, 151, 167, 155, 145, 151, 171, 159, 130, 145, 153, 152, 133, 159, 161, 163, 157, 161, 147, 159, 166, 182, 144, 174, 154, 146, 155, 159, 173, 141, 156, 161, 165, 146, 145, 167, 182, 143, 151, 157, 161, 149, 166, 145, 166, 167, 151, 153, 157, 181, 144, 157, 149, 152, 151, 153, 154, 160, 162, 149, 149, 162, 173, 164, 147,